In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tfx.components

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
tfx.__version__

'1.3.0'

In [3]:
data = pd.read_csv(os.path.join('data', 'completions.csv'))

In [4]:
data.loc[np.random.choice(data.index, 10_000), ['userId', 'itemId']].to_csv(os.path.join('data', 'sample', 'sample.csv'),index=False)

In [5]:
context= InteractiveContext()

In [6]:
SAMPLE_DATA = os.path.join('data', 'sample')

In [7]:
example_gen = tfx.components.CsvExampleGen(SAMPLE_DATA)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:116992,xor_checksum:1637322068,sum_checksum:1637322068"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [9]:
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

I1119 11:41:16.244827 13761 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'itemId',INT,required,,-
'userId',INT,required,,-


In [11]:
from tfx.dsl.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor

run_fn = 'rexify.pipeline.train.run_fn'
trainer_args = dict(
    run_fn=run_fn,
    examples=example_gen.outputs['examples'],
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=1000),
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor))
trainer = tfx.components.Trainer(**trainer_args)
context.run(trainer)

I1119 11:41:16.433411 13761 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I1119 11:41:16.447973 13761 rdbms_metadata_access_object.cc:686] No property is defined for the Type
2021-11-19 11:41:16.858778: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-19 11:41:16.858848: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (P640RF): /proc/driver/nvidia/version does not exist
2021-11-19 11:41:16.860205: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


   1/1000 [..............................] - ETA: 15:14 - loss: 3194.1899 - regularization_loss: 0.0000e+00 - total_loss: 3194.1899

2021-11-19 11:41:17.829028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1000/1000 [==============================] - 6s 5ms/step - loss: 504.3825 - regularization_loss: 0.0000e+00 - total_loss: 504.3825


2021-11-19 11:41:23.652993: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/Trainer/model/4/Format-Serving/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/Trainer/model/4/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 4
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/Trainer/model/4"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 5
        type_id: 21
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/Trainer/model_run/4"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [12]:
item_gen = tfx.components.CsvExampleGen(input_base='data/items').with_id('ItemGen')
context.run(item_gen)

ExecutionResult(
    component_id: ItemGen
    execution_id: 5
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/ItemGen/examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:38852,xor_checksum:1626106282,sum_checksum:1626106282"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
import json

with open('data/schema.json', 'r') as f:
    schema = json.load(f)

In [14]:
from rexify.pipeline import components as rexify_components

lookup_gen = rexify_components.LookupGen(
    examples=item_gen.outputs['examples'],
    model=trainer.outputs['model'],
    query_model='candidate_model',
    feature_key='itemId',
    schema=json.dumps(schema))

context.run(lookup_gen)

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/LookupGen/lookup_model/6/lookup_model/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/LookupGen/lookup_model/6/lookup_model/assets


ExecutionResult(
    component_id: LookupGen
    execution_id: 6
    outputs:
        lookup_model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 7
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/LookupGen/lookup_model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "lookup_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "LookupGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
scann_gen = rexify_components.ScaNNGen(
    candidates=item_gen.outputs['examples'],
    model=trainer.outputs['model'],
    lookup_model=lookup_gen.outputs['lookup_model'],
    schema=json.dumps(schema),
    feature_key='itemId')
context.run(scann_gen)

2021-11-19 11:41:30.716851: I scann/partitioning/partitioner_factory_base.cc:71] Size of sampled dataset for training partition: 4929
2021-11-19 11:41:30.732092: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:102] PartitionerFactory ran in 15.163532ms.


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/ScaNNGen/index/7/index/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/ScaNNGen/index/7/index/assets


ExecutionResult(
    component_id: ScaNNGen
    execution_id: 7
    outputs:
        index: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-11-19T11_41_08.554805-_yam7c6g/ScaNNGen/index/7"
        custom_properties {
          key: "name"
          value {
            string_value: "index"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ScaNNGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))